In [1]:
import sys, os
sys.path.append('../src')
from scipy.io import loadmat
from scipy.stats import wilcoxon, ranksums
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
from utils import temporal_sort

In [2]:
data = loadmat('data.mat')['data']
# data structure:
# 'zscore1' and 'zscore2' are the used keys:
# (1, 1)
# -> (number of birds = 8, 1) 
#   -> (time stamps = 120, number of neurons, number of trials)
data_corr = [d[0] for d in data['zscore1'][0,0]]
data_pert = [d[0] for d in data['zscore2'][0,0]]

In [3]:
data = loadmat('./data_raw.mat')['dataraw']['masterlist'][0,0]
# data structure:
# -> (number of birds = 6, 2 conditions = pre / post deafening)
#   -> Only [3,0] is useful; col 0 = singing
#      row 0 = denoised calcium signal
#      row 1 = deconvolved spikes
#      row 2 = raw calcium 
#      row 3 = zscored calcium 
#      the rest are blank
#     -> (time stamps = 120, number of neurons, number of trials)
idx = loadmat('./data_raw.mat')['dataraw']['neuronvec'][0,0]
# idx structure:
# -> (number of birds = 6, 1)
#   -> (registered neuron indices, 2 conditions = pre / post deafening)
data_predeaf, data_postdeaf = [], []
for i in range(data.shape[0]):
    data_predeaf.append(data[i,0][3,0][:,idx[i,0][:,0]-1,:]) 
    data_postdeaf.append(data[i,1][3,0][:,idx[i,0][:,1]-1,:])

In [4]:
tpre = 6
t0 = 60 # song onset
t1 = t0 + 12

In [5]:
def processed_activity(data):
    signals_mean, signals_se, mean_activity, se_activity = [], [], [], []   
    for signal in data:
        # (time, neurons, trials)
        signal = signal - signal[t0-tpre:t0].mean(axis=0, keepdims=True)

        time_avg = signal[t0:t1].mean(axis=0)
        
        signals_mean.append(signal[t0-tpre:t1].mean(axis=-1))
        signals_se.append(signal[t0-tpre:t1].std(axis=-1) / np.sqrt(signal.shape[-1]))
        mean_activity.append(time_avg.mean(axis=1))
        se_activity.append(time_avg.std(axis=1) / np.sqrt(time_avg.shape[1]))
        
    signals_mean = np.hstack(signals_mean)
    signals_se = np.hstack(signals_se)
    mean_activity, se_activity = np.hstack(mean_activity), np.hstack(se_activity)
    return signals_mean, signals_se, mean_activity, se_activity

In [6]:
# first two: mean and se of trial-averaged activity
# last two: mean and se of trial-averaged and window-averaged activity
signal_mean_corr, signal_se_corr, mean_corr, se_corr = processed_activity(data_corr)
signal_mean_pert, signal_se_pert, mean_pert, se_pert = processed_activity(data_pert)
signal_mean_predeaf, signal_se_predeaf, mean_predeaf, se_predeaf = processed_activity(data_predeaf)
signal_mean_postdeaf, signal_se_postdeaf, mean_postdeaf, se_postdeaf = processed_activity(data_postdeaf)

to_save = dict(
    signal_mean_corr=signal_mean_corr, signal_se_corr=signal_se_corr, 
    mean_corr=mean_corr, se_corr=se_corr,
    signal_mean_pert=signal_mean_pert, signal_se_pert=signal_se_pert, 
    mean_pert=mean_pert, se_pert=se_pert,
    signal_mean_predeaf=signal_mean_predeaf, signal_se_predeaf=signal_se_predeaf,
    mean_predeaf=mean_predeaf, se_predeaf=se_predeaf,
    signal_mean_postdeaf=signal_mean_postdeaf, signal_se_postdeaf=signal_se_postdeaf,
    mean_postdeaf=mean_postdeaf, se_postdeaf=se_postdeaf
)

for k, v in to_save.items():
    np.savetxt(f'./processed_data/{k}.csv', v, delimiter=',') 